In [1]:
!pip install sentence-transformers pypdf python-docx
!pip install streamlit



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.1 MB/s eta 0:00:00


# Upload Job Description

In [4]:
from google.colab import files

print("📄 Upload JOB DESCRIPTION file (TXT / PDF / DOCX):")
uploaded_job = files.upload()
job_description_file = list(uploaded_job.keys())[0]
print("Job description uploaded:", job_description_file)


📄 Upload JOB DESCRIPTION file (TXT / PDF / DOCX):


Saving Job_Description.txt to Job_Description (1).txt
Job description uploaded: Job_Description (1).txt


# Upload the Resumes

In [5]:
print("📂 Upload RESUME files (you can upload multiple):")
uploaded_resumes = files.upload()

resume_files = list(uploaded_resumes.keys())
print("Resumes uploaded:", resume_files)


📂 Upload RESUME files (you can upload multiple):


Saving CV 1.txt to CV 1 (1).txt
Saving CV 2.txt to CV 2 (1).txt
Saving CV 3.txt to CV 3 (1).txt
Resumes uploaded: ['CV 1 (1).txt', 'CV 2 (1).txt', 'CV 3 (1).txt']


In [15]:
def extract_text_from_pdf(pdf_path):
    text = ""
    reader = PdfReader(pdf_path)
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + " "
    return text

def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    return " ".join([p.text for p in doc.paragraphs])

def extract_text_from_txt(txt_path):
    with open(txt_path, "r", encoding="utf-8") as f:
        return f.read()

def extract_text(file_path):
    if file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".txt"):
        return extract_text_from_txt(file_path)
    else:
        return ""

job_text = extract_text(job_description_file)
print("✅ Job description extracted successfully!")

resumes_text = {}
for resume in resume_files:
    resumes_text[resume] = extract_text(resume)
print(f"✅ Extracted text from {len(resumes_text)} resumes.")

✅ Job description extracted successfully!
✅ Extracted text from 3 resumes.


In [19]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

def embed_text(text):
    return model.encode(text, convert_to_tensor=True)

def rank_resumes(resumes, job_description):
    job_embedding = embed_text(job_description)
    scores = {}
    for name, text in resumes.items():
        resume_embedding = embed_text(text)
        score = util.cos_sim(job_embedding, resume_embedding).item()
        scores[name] = score

    # Sort by descending
    ranked = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))
    return ranked

#  To Make sure job_description text is available
if job_text:
    ranked_resumes = rank_resumes(resumes_text, job_text)
    print("✅ Resumes ranked by semantic similarity!")
    for i, (resume, score) in enumerate(ranked_resumes.items(), 1):
        print(f"{i}. {resume} → Similarity Score: {score:.4f}")
else:
    print("⚠️ Job description text not found. Cannot rank resumes.")

✅ Resumes ranked by semantic similarity!
1. CV 1 (1).txt → Similarity Score: 0.7876
2. CV 2 (1).txt → Similarity Score: 0.5761
3. CV 3 (1).txt → Similarity Score: 0.5328
